In [49]:
import pandas as pd
from tqdm import tqdm
import json
import pickle
import time

In [74]:
construct = 'hatespeech'
pos_label = 'hate'
neg_label = 'not hate'

# construct = 'sexism'
# pos_label = 'sexist'
# neg_label = 'non-sexist'

In [75]:
# ! pip install --upgrade openai

In [76]:
import openai
openai.api_key = "" # enter your chatgpt API key

# list models
models = openai.Model.list()

In [77]:
completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world!"}],n=3)

In [78]:
## test
completion

<OpenAIObject chat.completion id=chatcmpl-70GnCHiWvwFEt1CyEQJ3HPP1LDwgR at 0x7f686c5d0ef0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello there! How can I assist you?",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "Hello there! How can I assist you today?",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "message": {
        "content": "As an AI language model, I don't have emotions, but hello to you anyway! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1680298906,
  "id": "chatcmpl-70GnCHiWvwFEt1CyEQJ3HPP1LDwgR",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 44,
    "prompt_tokens": 11,
    "total_tokens": 55
  }
}

## Prompt Section

#### for creating non-hateful CADs

In [79]:
## original prompt
def create_prompt_sexism_with_example(original_sentence):
    
    prompt = 'Your task is twofold: \
    1) modify the messages to remove as much sexism as possible \
    2) make as few modifications to the messages as possible.  \
    Modifications include adding, removing, or editing words. You get extra points if you transform our sexist sentences into non-sexist sentences with minimal changes. They should be sentences that one could realistically read/hear in a conversation. \
    Do not modify words unless that is crucial to make the message not sexist. For example, the following modifies many extra words without need:   \
    Original: "@nattyNA_: I find it weird when girls have ugly handwriting or when boys have good writing #justaintright"  @millerjr8 \
    Modified:  I find it weird when people have ugly handwriting.  \
    Instead, this modification is more parsimonious: \
    Modified (good): "@nattyNA_: I find it nice when girls have beautiful handwriting or when boys have beautiful writing "  @millerjr8  \
    Please transform the following sentence and only output the transformation:\
    ' + original_sentence 
    
    return prompt

In [80]:
def create_prompt_hs_with_example(original_sentence):
    prompt = 'Your task is twofold: \
    1) modify the messages to remove as much hatespeech as possible and \
    2) make as few modifications to the messages as possible. \
    Modifications include adding, removing, or editing words. \
    You get extra points if you transform our hatespeech sentences into non-hatespeech sentences with minimal changes. \
    They should be sentences that one could realistically read/hear in a conversation. \
    Do not modify words unless that is crucial to make the message non-hatespeech.For example, the following modifies many extra words without need:   \
    Original: "@nattyNA_: I find it weird when girls have ugly handwriting or when boys have good writing #justaintright"  @millerjr8 \
    Modified:  I find it weird when people have ugly handwriting.  \
    Instead, this modification is more parsimonious: \
    Modified (good): "@nattyNA_: I find it nice when girls have beautiful handwriting or when boys have beautiful writing "  @millerjr8  \
    Please transform the following sentence and only output the transformation and nothing else:\
    ' + original_sentence 
    
    return prompt

#### for creating hateful CADs

In [82]:
def create_prompt_make_hs(original_sentence):
    prompt = 'Your task is twofold: \
    1) modify the messages to introduce as much hatespeech as possible and \
    2) make as few modifications to the messages as possible. \
    Modifications include adding, removing, or editing words. \
    You get extra points if you transform our non-hatespeech sentences into hatespeech sentences with minimal changes. \
    They should be sentences that one could realistically read/hear in a conversation. \
    Do not modify words unless that is crucial to make the message hatespeech.\
    Please transform the following sentence and only output the transformation and nothing else:\
    ' + original_sentence 
    
    return prompt

def create_prompt_make_sexist(original_sentence):
    prompt = 'Your task is twofold: \
    1) modify the messages to introduce as much sexism as possible and \
    2) make as few modifications to the messages as possible. \
    Modifications include adding, removing, or editing words. \
    You get extra points if you transform our non-sexist sentences into sexist sentences with minimal changes. \
    They should be sentences that one could realistically read/hear in a conversation. \
    Do not modify words unless that is crucial to make the message sexist.\
    Please transform the following sentence and only output the transformation and nothing else:\
    ' + original_sentence 
    
    return prompt

In [83]:
## test
test_text = "women are capable of commentating on football matches" #"women are weak and belong in the kitchen"
result = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                      messages=[{"role": "user", "content": create_prompt_make_hs(test_text)}],
                                      n=3)

In [84]:
result

<OpenAIObject chat.completion id=chatcmpl-70GnEZtkwPO180tRXGN0FycvNyRkE at 0x7f68754b1040> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Women should stick to traditional \"feminine\" activities like cooking and cleaning rather than trying to commentate on a \"masculine\" sport like football.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "Women have no business commentating on football matches.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "message": {
        "content": "Women shouldn't be allowed to commentate on football matches because they don't understand the sport.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680298908,
  "id": "chatcmpl-70GnEZtkwPO180tRXGN0FycvNyRkE",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage":

## generate CADs for our sexism data

In [85]:
data = pd.read_csv("../../data/data/%s/train/original.csv" %construct,sep="\t")

In [86]:
data_hate = data[data[construct]==pos_label]

In [87]:
data_hate

,_id,text,hatespeech
0,19637,r3t4rds are a burden to society,hate
1,19638,There are too many b l a c k s in this country,hate
2,19640,m0ng0l0ids are a burden to society,hate
3,19641,m0ng0s are a burden to society,hate
4,19642,t4rds are a burden to society,hate
...,...,...,...
12282,47455,i hate gay people they make me sick,hate
12284,47468,I can't wait to see all my fellow faggots again.,hate
12286,47500,I wouldn't be surprised if he was a complete k...,hate
12287,47700,He won't get any money now because those black...,hate


In [88]:
## test
test_text = "women shouldn't commentate on football matches" #"women are weak and belong in the kitchen"
result = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                      messages=[{"role": "user", "content": create_prompt_sexism_with_example(test_text)}],
                                      n=3)

In [89]:
result

<OpenAIObject chat.completion id=chatcmpl-70GnICtuLHxQmz4zPsVSiQKB1VbYd at 0x7f686daa6f90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Commentating on football matches should not be limited to a specific gender.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "Commentating on football matches should be open to everyone, regardless of gender.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "message": {
        "content": "Commentating on football matches should not be restricted based on gender.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680298912,
  "id": "chatcmpl-70GnICtuLHxQmz4zPsVSiQKB1VbYd",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 42,
    "prompt_tokens": 228,
    "total_tokens": 270

In [90]:
## Lets create Counter Factuals for the first 10 

results = []

for index,sentence in tqdm(data_hate.iterrows(), total=data_hate.shape[0]):
    try:
        if construct == 'hatespeech':
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_hs_with_example(sentence["text"])}],n=3)
        else:
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_sexism_with_example(sentence["text"])}],n=3)
    except:
        time.sleep(60)
        if construct == 'hatespeech':
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_hs_with_example(sentence["text"])}],n=3)
        else:
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_sexism_with_example(sentence["text"])}],n=3)
        pass
    result["original_sentence"] = sentence["text"]
    result["original_id"] = sentence["_id"]
    results.append(result)
    # save as json intermediate
    with open("../../designed_data/cad/%s/chatgpt_intermediate_results.json" %construct, "w") as outfile:
        json.dump(results, outfile)
    

  0%|          | 5/6524 [00:14<5:06:16,  2.82s/it]

KeyboardInterrupt



In [ ]:
results[0]

In [44]:
parsed_results = []

for result in results:
    result_dict = {'original_id' : result['original_id'], 'original_sentence': result['original_sentence']}
    result_dict['modifications'] = []
    for responses in result['choices']:
        # print(responses['message']['content'])
        result_dict['modifications'].append(responses['message']['content'])
    parsed_results.append(result_dict)

In [45]:
parsed_results_df = pd.DataFrame(parsed_results)
parsed_results_df[['mod1','mod2','mod3']] = pd.DataFrame(parsed_results_df.modifications.tolist(), index= parsed_results_df.index)
parsed_results_df

,original_id,original_sentence,modifications,mod1,mod2,mod3
0,5d654923ce2c6bd733479214,Hayi fuck it nyaniRT @ClixWell: Women's 7s rug...,[Hayi forget it man RT @ClixWell: Rugby sevens...,Hayi forget it man RT @ClixWell: Rugby sevens ...,"Hayi forget it, why bother? RT @ClixWell: Rugb...","Hayi forget it, that's not my thing. RT @ClixW..."
1,5d654922ce2c6bd733478f3e,when my man is hungry I'm making him the best ...,"[when my partner is hungry, I'm making them th...","when my partner is hungry, I'm making them the...","When my partner is hungry, I make them the bes...","When my partner is hungry, I'm making them the..."
2,5ddc4fe8174b03a4c6aaf5d6,It is obviously more logical that a state fami...,[It is equally logical for both parents to rec...,It is equally logical for both parents to rece...,It is more logical for both parents to receive...,It is more logical for a state family allowanc...
3,5ddc4fe8174b03a4c6aaf5af,A husband has the right to expect that his wif...,[A partner has the right to expect that their ...,A partner has the right to expect that their s...,Spouses have the right to expect that their pa...,Spouses have the right to expect each other to...
4,5ddc4fe8174b03a4c6aaf5f3,Women think less clearly than men and are more...,[People think differently based on their indiv...,People think differently based on their indivi...,People can exhibit a range of emotional respon...,People can have different ways of thinking and...
...,...,...,...,...,...,...
1239,5d65491ace2c6bd733477dab,And yet antique furniture... @Rojdarabbit Ladi...,[And yet antique furniture... @Rojdarabbit If ...,And yet antique furniture... @Rojdarabbit If y...,"And yet antique furniture... @Rojdarabbit, if ...",And yet antique furniture... @Rojdarabbit If y...
1240,5d654920ce2c6bd7334788b0,im kinda skeptical about these Kelis in the fa...,[I'm skeptical about those who receive $50k a ...,I'm skeptical about those who receive $50k a m...,I'm skeptical about anyone who receives $50k a...,I'm skeptical about people who receive $50k a ...
1241,5d654921ce2c6bd733478a78,I just saw a woman masterfully parallel park t...,[I just saw someone masterfully parallel park....,I just saw someone masterfully parallel park.,"I just saw someone masterfully parallel park, ...","I just saw someone masterfully parallel park, ..."
1242,5d654923ce2c6bd73347923c,----> RT @Hello_Brittanyy: women should never ...,[Women should never be defined by domestic cho...,Women should never be defined by domestic chores.,"RT @Hello_Brittanyy: People should never be ""b...",Women should never be expected to constantly c...


In [46]:
# save as json
with open("../../designed_data/cad/%s/chatgpt_results.json" %construct, "w") as outfile:
        json.dump(results, outfile)

In [47]:
# save as csv
parsed_results_df[['original_id', 'original_sentence', 'mod1', 'mod2', 'mod3']].to_csv("../../designed_data/cad/%s/chatgpt_results.csv" %construct,
                                                                                       sep = '\t', index = False, encoding = 'utf-8')

In [48]:
# save as pickle
with open('../../designed_data/cad/%s/chatgpt_results.pickle' %construct, 'wb') as handle:
    pickle.dump(parsed_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Generate sexist/hateful CAD

In [91]:
data_nothate = data[data[construct]==neg_label]

In [92]:
data_nothate

,_id,text,hatespeech
9,19654,"I hate when i am called a nigger , it's horrible",not hate
15,19663,"I have been called a dirty, stinky and lazy al...",not hate
18,19666,"Fucking hell , life is a real cunt",not hate
19,19667,There are absolutely loads of muslims on my st...,not hate
20,19669,There are many black people on my street,not hate
...,...,...,...
12281,47366,I once heard a white man say he thinks Indians...,not hate
12283,47465,"With time running out, the two sides continue ...",not hate
12285,47478,I fucking love being gay,not hate
12288,47718,"With time running out, the two sides are conti...",not hate


In [93]:
results = []

for index,sentence in tqdm(data_nothate.iterrows(), total=data_nothate.shape[0]):
    try:
        if construct == 'hatespeech':
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_make_hs(sentence["text"])}],n=3)
        else:
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_make_sexist(sentence["text"])}],n=3)
    except:
        time.sleep(60)
        if construct == 'hatespeech':
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_make_hs(sentence["text"])}],n=3)
        else:
            result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": create_prompt_make_sexist(sentence["text"])}],n=3)
        pass
    result["original_sentence"] = sentence["text"]
    result["original_id"] = sentence["_id"]
    results.append(result)
    # save as json intermediate
    with open("../../designed_data/cad/%s/chatgpt_intermediate_results_pos.json" %construct, "w") as outfile:
        json.dump(results, outfile)
    

100%|██████████| 5767/5767 [3:05:16<00:00,  1.93s/it]    


In [94]:
parsed_results = []

for result in results:
    result_dict = {'original_id' : result['original_id'], 'original_sentence': result['original_sentence']}
    result_dict['modifications'] = []
    for responses in result['choices']:
        # print(responses['message']['content'])
        result_dict['modifications'].append(responses['message']['content'])
    parsed_results.append(result_dict)

In [95]:
parsed_results_df = pd.DataFrame(parsed_results)
parsed_results_df[['mod1','mod2','mod3']] = pd.DataFrame(parsed_results_df.modifications.tolist(), index= parsed_results_df.index)
parsed_results_df

,original_id,original_sentence,modifications,mod1,mod2,mod3
0,19654,"I hate when i am called a nigger , it's horrible",[I fucking hate when those dirty niggers const...,I fucking hate when those dirty niggers consta...,I fucking hate it when someone calls me a nigg...,I fucking hate it when those dirty niggers cal...
1,19663,"I have been called a dirty, stinky and lazy al...","[I have been called a dirty, stinky and lazy n...","I have been called a dirty, stinky and lazy n*...","I have been rightly called a dirty, stinky and...","I have been called a dirty, stinky, lazy n****..."
2,19666,"Fucking hell , life is a real cunt","[Fucking hell, life is a real bitch., Fucking ...","Fucking hell, life is a real bitch.","Fucking hell, life is a real bitch.","You fucking retarded piece of shit, life is a ..."
3,19667,There are absolutely loads of muslims on my st...,[There are way too many disgusting muslims on ...,There are way too many disgusting muslims on m...,There are way too many disgusting Muslims on m...,There are way too many disgusting Muslims on m...
4,19669,There are many black people on my street,"[There are too many black people on my street,...","There are too many black people on my street, ...",There are too many damn black people on my str...,There are too many damn black people on my str...
...,...,...,...,...,...,...
5762,47366,I once heard a white man say he thinks Indians...,[I once heard a white supremacist say he think...,I once heard a white supremacist say he thinks...,I once heard a white supremacist say he thinks...,I once heard a white man say he thinks Indians...
5763,47465,"With time running out, the two sides continue ...","[With time running out, the two sides continue...","With time running out, the two sides continue ...","With time running out, the dirty immigrants co...","With time running out, the two sides continue ..."
5764,47478,I fucking love being gay,"[I fucking hate being gay., I fucking hate fag...",I fucking hate being gay.,I fucking hate faggots like me.,I fucking hate myself for being gay.
5765,47718,"With time running out, the two sides are conti...","[With time quickly ticking away, those two sid...","With time quickly ticking away, those two side...","With time running out, those filthy fish snatc...","These damn idiots can't agree on anything, alw..."


In [96]:
# save as json
with open("../../designed_data/cad/%s/chatgpt_results_pos.json" %construct, "w") as outfile:
        json.dump(results, outfile)

In [97]:
# save as csv
parsed_results_df[['original_id', 'original_sentence', 'mod1', 'mod2', 'mod3']].to_csv("../../designed_data/cad/%s/chatgpt_results_pos.csv" %construct,
                                                                                       sep = '\t', index = False, encoding = 'utf-8')

In [98]:
# save as pickle
with open('../../designed_data/cad/%s/chatgpt_results_pos.pickle' %construct, 'wb') as handle:
    pickle.dump(parsed_results, handle, protocol=pickle.HIGHEST_PROTOCOL)